In [25]:
#Connection aux API et à la bdd neo4j
print('Connection aux API et à la bdd neo4j')
%run ../authentification.ipynb

#Importation librairies plotly
import plotly.plotly as py
import plotly.graph_objs as go

#Importation librairies pandas
from pandas import DataFrame
import pandas as pd
import pandas_datareader.data as web

#Importation librairies matplotlib
import matplotlib.pyplot as plt

#Importation librairies textblob
from textblob import TextBlob

#Autres
import re
from datetime import datetime


import operator 
import json
from collections import Counter
import re
import nltk
#######  TRES IMPORTANT: decommenter la ligne ci dessous et download corpora nltk ############
# nltk.download()
from nltk.corpus import stopwords
import string


Connection aux API et à la bdd neo4j
Authentification à l'api twitter réussie
Connexion à neo4j réussie
Authentification à l'api plotly réussie
Merci de vérifier que vous avez les modules pythons correctement installés. faire notamment attention à la librairie nltk à installer avec un nltk.download()


In [26]:
#Methode de Sentiment Analysis
def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def analize_sentiment(tweet):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    analysis = TextBlob(clean_tweet(tweet))
    return analysis.sentiment.polarity*10



numbers=[]

i=0
while i <= 2018: # Tant que i est strictement inférieure à 10
    numbers.append(str(i))
    i += 1
#Methodes pour le comptage des occurences des mots
alphabet= list(string.ascii_lowercase)
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['httpstcod','korean','httpstcos', 'donald','jong','america','american','north','korea',
                                                   'httpstcob','trumpkim','northkorea',
                                                   'trumps','donald','singapore','donaldtrump',
                                                   'jongun','jong','un','north','korea','america',
                                                   'rt', 'via','i','the','amp','realdonaldtrump','httpstco',
                                                   'trump','kim','trumpkimsummit','meeting','summit',
                                                   'singaporemeeting', 'kimjongun','donaldtrump','us', 
                                                   'httpstcoku','03'] +list(numbers) + alphabet

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)

def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens








In [50]:
%%HTML
<h1>Analyse de la tendance générale</h1>
<h3 style="color:purple;" >1) 20 Hashtags les plus utilisés dans les tweets collectés</h3>
<p>Par choix, nous avons limité les hashtags ne contenant pas les mots clés suivants:</p>
    <UL>
        <LI>Trump</LI>
        <LI>Kim</LI>
        <LI>Summit</LI>
        <LI>Trumpkim</LI>
        <LI>TrumpKimSummit</LI>
        <LI>TrumpKimMeeting</LI>
        <LI>SingaporeMeeting</LI>
        <LI>SingaporeSummit</LI>
        <LI>...</LI>
    </UL>

In [28]:
queryImportantPeople='MATCH (t:Tweet)-[re:HAS]->(h:Hashtag) WHERE NOT ((lower(h.tag) = "trump") OR (lower(h.tag)="trumpkim") OR (lower(h.tag)="trumpkimsummit") OR (lower(h.tag)="kim") OR (lower(h.tag)="trumpkimmeeting") OR (lower(h.tag)="singaporesummit") OR (lower(h.tag)="singaporemeeting") OR (lower(h.tag)="kimjongun") OR (lower(h.tag)="donaldtrump")) return h.tag as Tag, count(h.tag) as OccurenceTags ORDER BY OccurenceTags DESC LIMIT 20'

dfImportantPeople= pd.DataFrame(sgraph.data(queryImportantPeople))


#Creation du graphique
labels = dfImportantPeople['Tag']
values = dfImportantPeople['OccurenceTags']
colors = ['#eeaba5', '#da4236', '#801b13', '#b2dbc6','#4cab7c','#008744']

tracetags = go.Pie(labels=labels, values=values,
               hoverinfo='label+percent', textinfo='value',
               textfont=dict(size=20),
               marker=dict(colors=colors,
                           line=dict(color='#000000', width=2)))

py.iplot([tracetags], filename='tracetagsstyled_pie_chart')
    


In [29]:
%%HTML
<h3 style="color:purple;"> 2) Sentiment Analysis des tweets postés </h3>


In [30]:
queryTweetsPosted="MATCH (t:Tweet) return lower(t.text)" 
datasent=sgraph.run(queryTweetsPosted)


dfPostedTweets = pd.DataFrame(data=[(tweet[0],analize_sentiment(tweet[0])) for tweet in datasent], columns=['Tweets', 'SA'])

# We construct lists with classified tweets:

slightlypos_tweets = [ tweet for index, tweet in enumerate(dfPostedTweets['Tweets']) if dfPostedTweets['SA'][index] >= 0 and dfPostedTweets['SA'][index] < 3 ]
mostlypos_tweets = [ tweet for index, tweet in enumerate(dfPostedTweets['Tweets']) if dfPostedTweets['SA'][index] >= 3 and dfPostedTweets['SA'][index] < 6]
complypos_tweets = [ tweet for index, tweet in enumerate(dfPostedTweets['Tweets']) if dfPostedTweets['SA'][index] >= 6]
slightlyneg_tweets = [ tweet for index, tweet in enumerate(dfPostedTweets['Tweets']) if dfPostedTweets['SA'][index] < 0 and dfPostedTweets['SA'][index] >= (-3)]
mostlyneg_tweets = [ tweet for index, tweet in enumerate(dfPostedTweets['Tweets']) if dfPostedTweets['SA'][index] < (-3) and dfPostedTweets['SA'][index] > (-6)]
complyneg_tweets = [ tweet for index, tweet in enumerate(dfPostedTweets['Tweets']) if dfPostedTweets['SA'][index] <=(-6)]


#Creation du graphique
labels = ['SlightlyNegative', 'MostlyNegative' , 'CompletlyNegative','SlightlyPositive', 'MostlyPositive' , 'CompletlyPositive' ]
values = [len(slightlyneg_tweets), len(mostlyneg_tweets), len(complyneg_tweets), len(slightlypos_tweets), len(mostlypos_tweets), len(complypos_tweets)  ]
colors = ['#eeaba5', '#da4236', '#801b13', '#b2dbc6','#4cab7c','#008744']

traceSA = go.Pie(labels=labels, values=values,
               hoverinfo='label+percent', textinfo='value',
               textfont=dict(size=20),
               marker=dict(colors=colors,
                           line=dict(color='#000000', width=2)))

py.iplot([traceSA], filename='SATGstyled_pie_chart')
    

In [31]:
%%HTML
<h3 style="color:purple;" >3) Analyse du nombre d'occurences de mots dans les tweets postés</h3>



In [32]:

count_all = Counter()

for line in dfPostedTweets['Tweets']:
    
        # Create a list with all the terms
    terms_all = [term for term in preprocess(line) if term not in stop]
    #terms_all = [term for term in preprocess(line)]
        # Update the counter
    count_all.update(terms_all)
    # Print the first 5 most frequent words


data = pd.DataFrame(data=count_all.most_common(30), columns=['mots','occurrence'])
display(data)

,mots,occurrence
0,president,2772
1,got,2758
2,gag,2391
3,like,2296
4,great,1994
5,nuclear,1991
6,peace,1900
7,people,1840
8,whats,1821
9,favorite,1671


In [33]:
%%HTML
<h3 style="color:purple;" >4) Support les plus utilisés pour les tweets collectés</h3>
<p>Par choix, nous avons limité les supports aux 10 les plus utilisés</p>

In [34]:
querysource="MATCH (t:Tweet) return t.source, count(t)  ORDER BY count(t) DESC LIMIT 10"

dfsource= pd.DataFrame(sgraph.data(querysource))

labels = dfsource['t.source']
values = dfsource['count(t)']
colors = ['#FEBFB3', '#E1396C', '#96D38C', '#D0F9B1']

tracesupp = go.Pie(labels=labels, values=values,
               hoverinfo='label+percent', textinfo='value',
               textfont=dict(size=20),
               marker=dict(colors=colors,
                           line=dict(color='#000000', width=2)))

py.iplot([tracesupp], filename='suppstyled_pie_chart')


In [35]:
%%HTML
<h3 style="color:purple;" >5) Analyse chronologique des nombres de tweets postés</h3>



In [36]:
querydatetweet="MATCH (n:Tweet) RETURN left(n.date,16) as datetweet , count(*) order by datetweet"

dfdatetweet= pd.DataFrame(sgraph.data(querydatetweet))

trace = go.Scatter(x=dfdatetweet["datetweet"],
                   y=dfdatetweet['count(*)'])

data = [trace]
layout = dict(
    title='Nombre de Tweets postés chronologiquement (UTC Time)',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1 mois',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6 mois',
                     step='month',
                     stepmode='backward'),
                dict(count=1,
                    label='Janvier -> Aujourd"hui',
                    step='year',
                    stepmode='todate'),
                dict(count=1,
                    label='1 an',
                    step='year',
                    stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(),
        type='date'
    )
)

fig = dict(data=data, layout=layout)
py.iplot(fig)

In [37]:
%%HTML
<h1>Analyse des Tweets les plus influents<h1>
<h2>1) Analyse des tweets des personnes ayant le plus de followers</h2>
<h3 style="color:purple;"> 1.A) Analyse des Tags utilisés par les Users des tweets collectés ayant le plus de followers</h3>
<p>Par choix, nous avons limité l'analyse au nombre de 20 users et avons supprimé les tags suivants:</p>
    <UL>
        <LI>Trump</LI>
        <LI>TrumpKim</LI>
        <LI>TrumpKimSummit</LI>
        <LI>TrumpKimMeeting</LI>
        <LI>SingaporeSummit</LI>
        <LI>SingaporeMeeting</LI>
        <LI>Trump</LI>
        <LI>kimjongun</LI>
        <LI>DonaldTrump</LI>
        <LI>Kim</LI>
        <LI>Summit</LI>
    </UL>

In [38]:
queryImportantPeople='MATCH (u:User)-[r:Posted]->(t:Tweet)-[re:HAS]->(h:Hashtag) WHERE NOT ((lower(h.tag) = "trump") OR (lower(h.tag)="trumpkim") OR (lower(h.tag)="trumpkimsummit") OR (lower(h.tag)="kim") OR (lower(h.tag)="trumpkimmeeting") OR (lower(h.tag)="singaporesummit") OR (lower(h.tag)="singaporemeeting") OR (lower(h.tag)="kimjongun") OR (lower(h.tag)="donaldtrump")) return u.screen_name as UserScreenName,u.followers_count as UserFollowersCount,collect(DISTINCT h.tag) as ListeTags ORDER BY toInt(u.followers_count) DESC LIMIT 20'

dfImportantPeople= pd.DataFrame(sgraph.data(queryImportantPeople))
display(dfImportantPeople)



,ListeTags,UserFollowersCount,UserScreenName
0,[Backstory],19799694,Reuters
1,"[Singapore, NorthKorea, China, SouthKorea, Pom...",1013526,STcom
2,[CNN],986341,geoffdeweaver
3,"[WhatsOnYourCan, WhatsOnYourCup, MofayaBranded...",722064,djsbu
4,[Rodman],688294,SABreakingNews
5,"[, Singaporean]",597481,globaltimesnews
6,[Backstory],577791,ReutersChina
7,"[China, Pakistan, India]",466851,FinancialXpress
8,"[NorthKorea, denuclearization, US, Sanctions, ...",425608,Fnewsenglish
9,"[WednesdayWisdom, RETWEET, BuildThatWall]",317562,WeSupport45


In [39]:
%%HTML
<h3 style="color:purple;"> 1.B) Sentiment analysis des tweets postés par les Users ayant le plus de followers</h3>

In [40]:
queryImportantPeopleTweets="MATCH (u:User)-[r:Posted]->(t:Tweet) WHERE toInt(u.followers_count)>10000 return lower(t.text)" 
datasent=sgraph.run(queryImportantPeopleTweets)


dfImportantTweets = pd.DataFrame(data=[(tweet[0],analize_sentiment(tweet[0])) for tweet in datasent], columns=['Tweets', 'SA'])

# We construct lists with classified tweets:

slightlypos_tweets = [ tweet for index, tweet in enumerate(dfImportantTweets['Tweets']) if dfImportantTweets['SA'][index] >= 0 and dfImportantTweets['SA'][index] < 3 ]
mostlypos_tweets = [ tweet for index, tweet in enumerate(dfImportantTweets['Tweets']) if dfImportantTweets['SA'][index] >= 3 and dfImportantTweets['SA'][index] < 6]
complypos_tweets = [ tweet for index, tweet in enumerate(dfImportantTweets['Tweets']) if dfImportantTweets['SA'][index] >= 6]
slightlyneg_tweets = [ tweet for index, tweet in enumerate(dfImportantTweets['Tweets']) if dfImportantTweets['SA'][index] < 0 and dfImportantTweets['SA'][index] >= (-3)]
mostlyneg_tweets = [ tweet for index, tweet in enumerate(dfImportantTweets['Tweets']) if dfImportantTweets['SA'][index] < (-3) and dfImportantTweets['SA'][index] > (-6)]
complyneg_tweets = [ tweet for index, tweet in enumerate(dfImportantTweets['Tweets']) if dfImportantTweets['SA'][index] <=(-6)]


#Creation du graphique
labels = ['SlightlyNegative', 'MostlyNegative' , 'CompletlyNegative','SlightlyPositive', 'MostlyPositive' , 'CompletlyPositive' ]
values = [len(slightlyneg_tweets), len(mostlyneg_tweets), len(complyneg_tweets), len(slightlypos_tweets), len(mostlypos_tweets), len(complypos_tweets)  ]
colors = ['#eeaba5', '#da4236', '#801b13', '#b2dbc6','#4cab7c','#008744']

traceSAIT = go.Pie(labels=labels, values=values,
               hoverinfo='label+percent', textinfo='value',
               textfont=dict(size=20),
               marker=dict(colors=colors,
                           line=dict(color='#000000', width=2)))

py.iplot([traceSAIT], filename='SAITstyled_pie_chart')
    
    

In [41]:
%%HTML
<h3 style="color:purple;"> 1.C) Analyse du nombre d'occurences de mots dans les tweets postés</h3>

In [42]:
count_all = Counter()

for line in dfImportantTweets['Tweets']:
    
        # Create a list with all the terms
    terms_all = [term for term in preprocess(line) if term not in stop]
    #terms_all = [term for term in preprocess(line)]
        # Update the counter
    count_all.update(terms_all)
    # Print the first 5 most frequent words


data = pd.DataFrame(data=count_all.most_common(30), columns=['mots','occurrence'])
display(data)

,mots,occurrence
0,president,243
1,would,205
2,peace,186
3,obama,158
4,potus,151
5,days,146
6,one,139
7,cnn,124
8,like,121
9,historic,112


In [43]:
%%HTML
<h1>Analyse des Tweets les plus influents<h1>
<h2>2) Analyse des tweets postés par les médias</h2>
<h3 style="color:purple;">2.A)Analyse des Tags utilisés par les Médias</h3>
<p>Par choix, nous avons limité l'analyse au nombre de 20 users et avons supprimé les tags suivants:</p>
    <UL>
        <LI>Trump</LI>
        <LI>TrumpKim</LI>
        <LI>TrumpKimSummit</LI>
        <LI>TrumpKimMeeting</LI>
        <LI>SingaporeSummit</LI>
        <LI>SingaporeMeeting</LI>
        <LI>Trump</LI>
        <LI>kimjongun</LI>
        <LI>DonaldTrump</LI>
        <LI>Kim</LI>
        <LI>Summit</LI>
    </UL>

In [44]:
queryMedias='MATCH (u:User)-[r:Posted]->(t:Tweet)-[re:HAS]->(h:Hashtag) WHERE lower(u.screen_name) CONTAINS "news" AND NOT ((lower(h.tag) = "trump") OR (lower(h.tag)="trumpkim") OR (lower(h.tag)="trumpkimsummit") OR (lower(h.tag)="kim") OR (lower(h.tag)="trumpkimmeeting") OR (lower(h.tag)="singaporesummit") OR (lower(h.tag)="singaporemeeting") OR (lower(h.tag)="kimjongun") OR (lower(h.tag)="donaldtrump")) return u.screen_name as UserScreenName,u.followers_count as UserFollowersCount,collect(DISTINCT h.tag) as ListeTags ORDER BY toInt(u.followers_count) DESC LIMIT 20'

dfMedias= pd.DataFrame(sgraph.data(queryMedias))
display(dfMedias)



,ListeTags,UserFollowersCount,UserScreenName
0,[Rodman],688294,SABreakingNews
1,"[, Singaporean]",597481,globaltimesnews
2,"[NorthKorea, denuclearization, US, Sanctions, ...",425608,Fnewsenglish
3,[US],157409,arabnews
4,[tariffs],112640,newschannelnine
5,"[NorthKorea, Nuclearthreat]",95020,newsonepk
6,[NorthKorea],86352,CBNNews
7,[morintoon],62921,DannyShookNews
8,"[worldcup, FFRNN, BRAINWASHER]",50718,FFRNewsNet1
9,[Russia],41874,JuliaDavisNews


In [45]:
%%HTML
<h3 style="color:purple;">2.B) Sentimental Analysis des tweets postés par les Médias</h3>


In [47]:

queryMediasTweets='MATCH (u:User)-[r:Posted]->(t:Tweet) WHERE lower(u.screen_name) CONTAINS "news" return lower(t.text)' 
datasent=sgraph.run(queryMediasTweets)


dfMediasTweets = pd.DataFrame(data=[(tweet[0],analize_sentiment(tweet[0])) for tweet in datasent], columns=['Tweets', 'SA'])

# We construct lists with classified tweets:

slightlypos_tweets = [ tweet for index, tweet in enumerate(dfMediasTweets['Tweets']) if dfMediasTweets['SA'][index] >= 0 and dfMediasTweets['SA'][index] < 3 ]
mostlypos_tweets = [ tweet for index, tweet in enumerate(dfMediasTweets['Tweets']) if dfMediasTweets['SA'][index] >= 3 and dfMediasTweets['SA'][index] < 6]
complypos_tweets = [ tweet for index, tweet in enumerate(dfMediasTweets['Tweets']) if dfMediasTweets['SA'][index] >= 6]
slightlyneg_tweets = [ tweet for index, tweet in enumerate(dfMediasTweets['Tweets']) if dfMediasTweets['SA'][index] < 0 and dfMediasTweets['SA'][index] >= (-3)]
mostlyneg_tweets = [ tweet for index, tweet in enumerate(dfMediasTweets['Tweets']) if dfMediasTweets['SA'][index] < (-3) and dfMediasTweets['SA'][index] > (-6)]
complyneg_tweets = [ tweet for index, tweet in enumerate(dfMediasTweets['Tweets']) if dfMediasTweets['SA'][index] <=(-6)]


#Creation du graphique
labels = ['SlightlyNegative', 'MostlyNegative' , 'CompletlyNegative','SlightlyPositive', 'MostlyPositive' , 'CompletlyPositive' ]
values = [len(slightlyneg_tweets), len(mostlyneg_tweets), len(complyneg_tweets), len(slightlypos_tweets), len(mostlypos_tweets), len(complypos_tweets)  ]
colors = ['#eeaba5', '#da4236', '#801b13', '#b2dbc6','#4cab7c','#008744']

traceSAM = go.Pie(labels=labels, values=values,
               hoverinfo='label+percent', textinfo='value',
               textfont=dict(size=20),
               marker=dict(colors=colors,
                           line=dict(color='#000000', width=2)))

py.iplot([traceSAM], filename='SAMstyled_pie_chart')
    
    

In [48]:
%%HTML
<h3 style="color:purple;">2.C) Analyse du nombre d'occurences de mots des tweets postés par les Médias</h3>


In [49]:
count_all = Counter()

for line in dfMediasTweets['Tweets']:
    
        # Create a list with all the terms
    terms_all = [term for term in preprocess(line) if term not in stop]
    #terms_all = [term for term in preprocess(line)]
        # Update the counter
    count_all.update(terms_all)
    # Print the first 5 most frequent words


data = pd.DataFrame(data=count_all.most_common(30), columns=['mots','occurrence'])
display(data)


,mots,occurrence
0,says,13
1,news,12
2,president,11
3,historic,10
4,great,8
5,watch,7
6,time,7
7,latest,7
8,new,7
9,tictoc,7
